# Primer modelo automático usando TPOT

## Importamos las librerías

In [1]:
import numpy as np
import pandas as pd
from tpot import TPOTClassifier

from sklearn.model_selection import train_test_split

## Importamos los datos con pandas

In [2]:
# Cargando los datos
datos_titanic = pd.read_csv('./titanic_train.csv')
entrenamiento, pruebas = train_test_split(datos_titanic,test_size=0.3)


In [3]:
entrenamiento.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,623.000000,623.000000,623.000000,506.000000,623.000000,623.000000,623.000000
mean,442.523274,0.380417,2.308186,29.966403,0.484751,0.353130,30.941980
std,254.888354,0.485880,0.830322,14.072138,1.009286,0.792155,42.365516
min,1.000000,0.000000,1.000000,0.670000,0.000000,0.000000,0.000000
25%,223.000000,0.000000,2.000000,21.000000,0.000000,0.000000,7.925000
50%,439.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.108300
75%,664.500000,1.000000,3.000000,39.000000,1.000000,0.000000,30.070800
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,263.000000


In [4]:
entrenamiento.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
360,361,0,3,"Skoog, Mr. Wilhelm",male,40.0,1,4,347088,27.9000,NaN,S
798,799,0,3,"Ibrahim Shawah, Mr. Yousseff",male,30.0,0,0,2685,7.2292,NaN,C
372,373,0,3,"Beavan, Mr. William Thomas",male,19.0,0,0,323951,8.0500,NaN,S
409,410,0,3,"Lefebre, Miss. Ida",female,NaN,3,1,4133,25.4667,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


## Hacemos una "limpieza" de nuestro datos antes de hacer el modelo

In [5]:
combine = [entrenamiento, pruebas]

In [6]:
# Convert string values 'male' and 'female' to int values
sex_mapping = {'male': 0, 'female': 1}
entrenamiento['Sex'] = entrenamiento['Sex'].map(sex_mapping)
pruebas['Sex'] = pruebas['Sex'].map(sex_mapping)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [7]:
calculo_edades = np.zeros((2,3))

In [8]:
for dataset in combine:
    for sex in range(0, 2):
        for pclass in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == sex) & (dataset['Pclass'] == pclass+1)]['Age'].dropna()
            age_guess = guess_df.median()
            calculo_edades[sex, pclass] = int(age_guess/0.5 + 0.5) * 0.5
    
    for sex in range(0, 2):
        for pclass in range(0, 3):
            dataset.loc[(dataset.Age.isnull()) & (dataset.Sex == sex) &(dataset.Pclass == pclass+1),'Age'] = calculo_edades[sex, pclass]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [9]:
entrenamiento = entrenamiento.drop(['Ticket', 'Cabin', 'Name', 'PassengerId', 'SibSp', 'Parch', 'Embarked'], axis=1)
pruebas = pruebas.drop(['Ticket', 'Cabin', 'Name', 'SibSp', 'Parch', 'Embarked'], axis=1)

X_train = entrenamiento.drop('Survived', axis=1)
Y_train = entrenamiento['Survived']
X_test  = pruebas.drop(["PassengerId","Survived"], axis=1)

## Creamos y entrenamos nuestro modelo

In [10]:
tpot = TPOTClassifier(verbosity=2, max_time_mins=2)

In [11]:
tpot.fit(X_train, Y_train)

Generation 1 - Current best internal CV score: 0.8283096774193549

2.00 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: GradientBoostingClassifier(PCA(input_matrix, iterated_power=3, svd_solver=randomized), learning_rate=0.01, max_depth=4, max_features=0.2, min_samples_leaf=18, min_samples_split=20, n_estimators=100, subsample=1.0)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=100,
               max_eval_time_mins=5, max_time_mins=2, memory=None,
               mutation_rate=0.9, n_jobs=1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=100,
               random_state=None, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import PCA

In [13]:
nuevo_modelo = GradientBoostingClassifier(PCA(PCA(iterated_power=10, svd_solver='randomized'), iterated_power=6, svd_solver='randomized'), learning_rate=0.1, max_depth=5, max_features=0.8500000000000001, min_samples_leaf=16, min_samples_split=9, n_estimators=100, subsample=0.5)

In [14]:
nuevo_modelo.fit(X_train, Y_train)

TypeError: unsupported format string passed to PCA.__format__

## Predecimos con nuestro árbol y la tasa de exactitud

In [ ]:
Y_pred = nuevo_modelo.predict(X_test)

In [ ]:
Y_pred

In [ ]:
decision_tree.score(X_train, Y_train)

In [ ]:
tpot.export("algo.py")